In [1]:
import numpy as np
from ADAL import ADAL
from IRWA import IRWA

In [2]:
from qpsolvers import solve_qp

M = np.array([[1.0, 2.0, 0.0], [-8.0, 3.0, 2.0], [0.0, 1.0, 1.0]])
P = M.T @ M  # this is a positive definite matrix
q = np.array([3.0, 2.0, 3.0]) @ M
G = np.array([[1.0, 2.0, 1.0], [2.0, 0.0, 1.0], [-1.0, 2.0, -1.0]])
h = np.array([3.0, 2.0, -2.0])
A = np.array([[1.0, 1.0, 1.0]])
b = np.array([1.0])

x = solve_qp(P, q, G, h, A, b, solver="osqp")
print(f"QP solution: {x = }")

QP solution: x = array([ 0.3076548 , -0.69232615,  1.38448776])


/opt/homebrew/lib/python3.10/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:24: UserWarning: Converted P to scipy.sparse.csc.csc_matrix
For best performance, build P as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:24: UserWarning: Converted G to scipy.sparse.csc.csc_matrix
For best performance, build G as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:24: UserWarning: Converted A to scipy.sparse.csc.csc_matrix
For best performance, build A as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(


In [3]:
x = ADAL(A,G,-b,-h,q,P,np.array([1.0, 1.0, 1.0]),np.array([1.0, 1.0, 1.0,1.0]),np.array([1.0, 1.0, 1.,1.0]),1,1,1)
print(x)

[ 0.30769214 -0.69230883  1.38461612]


In [4]:
x = IRWA(A,G,-b,-h,q,P,np.array([1,1,1]),epsilon0=1e-100*np.array([1.0, 1.0, 1.0,1.0]),eta= 0.7,gamma=1/6,M= 10000,sigma=1e-6,sigma_prime=1e-8)
print(x)


[ 0.29339951 -0.67094529  1.09047864]
